# Wind Data From NOAA's Rapid Refresh
---

**NOAA**: National Oceanic and Atmospheric Administration

**RAP**: Rapid Refresh  
Information can be found at the following url:
https://www.ncdc.noaa.gov/data-access/model-data/model-datasets/rapid-refresh-rap

Data can be retrieved using the NetCDF Subset Service (NCSS). Information on this protocol are available at: https://www.unidata.ucar.edu/software/thredds/current/tds/reference/NetcdfSubsetServiceReference.html

The Rapid Refresh (RAP) numerical weather model is run by the National Centers for Environmental Prediction (NCEP), which is part of of the NOAA. Multiple data sources go into the generation of RAP model: commercial aircraft weather data, balloon data, radar data, surface observations, and satellite data. The model generates data down to a 13x13 squre kilometer resolution grid every hour. 

In [1]:
from powersimdata.input.grid import Grid
from prereise.gather.winddata.rap import rap, impute, helpers 

import pandas as pd
from matplotlib import pyplot

## 1. Wind Farms in Network

In [2]:
grid = Grid(['Texas','Western'])
wind_farm = grid.plant.groupby('type').get_group('wind')

--> Loading Texas+Western interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


In [3]:
wind_farm.head()

,bus_id,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,mu_Qmax,mu_Qmin,GenMWMax,GenMWMin,type,interconnect,lat,lon,zone_id,zone_name
plant_id,,,,,,,,,,,,,,,,,,,,,
10397,2010691,59.72,21.07,21.07,-14.24,1.0061,98.92,1,59.72,59.72,...,0.0,0.0,98.900002,16.648942,wind,Western,45.813100,-120.347500,201,Washington
10400,2010699,79.42,-14.50,21.45,-14.50,1.0019,120.64,1,79.42,79.42,...,0.0,0.0,100.699997,13.150001,wind,Western,47.135600,-120.687200,201,Washington
10401,2010703,136.30,29.03,29.03,-19.63,1.0623,174.64,1,136.30,136.30,...,0.0,0.0,136.300004,43.062568,wind,Western,45.879700,-120.807200,201,Washington
10428,2010746,66.35,-12.96,19.17,-12.96,1.0205,98.26,1,66.35,66.35,...,0.0,0.0,89.999998,19.930001,wind,Western,46.954700,-120.181900,201,Washington
10442,2010768,212.51,56.83,56.83,-38.42,1.0299,301.55,1,212.51,212.51,...,0.0,0.0,266.799998,122.148347,wind,Western,46.421111,-118.026944,201,Washington


In [4]:
print("There are %d wind farms in the Western + Texas interconnection." % len(wind_farm))

There are 330 wind farms in the Western + Texas interconnection.


## 2. Downloading Data from NCEP's Server
Only few days are considered here. Note that some files on the server may be missing.

In [5]:
data, missing = rap.retrieve_data(wind_farm, start_date='2016-01-01', end_date='2016-01-10')

100%|██████████| 240/240 [09:00<00:00,  2.61s/it]


In [ ]:
print("There are %d missing files." % len(missing))

In [ ]:
data.head()

## 3. Plot Data
One wind farm is selected. The U and V components of the wind speed (in m/s) 80-m above ground along with the power output (in MW) are then plotted. One can see the missing data. Note that there are 242 files (hour equivalent) missing in 2016.

In [ ]:
data_oneplant = data[data.plantID == 38]

In [ ]:
fig = pyplot.figure(figsize=(16,12))
for i, var in enumerate(['U','V','Pout']):
    ax = pyplot.subplot(3, 1, i+1, sharex=True)
    data_oneplant.plot(x='ts', y=var, ax=ax[i], legend=False, fontsize=15, lw=2)
    ax.set_xlabel('')
    ax.set_ylabel(var, fontsize=20)

pyplot.show()

## 4. Impute Missing Data
The missing data are imputed inplace using a simple procedure. The plot below shows the imputed data.

In [ ]:
impute.simple(data, wind_farm, inplace=True)

In [ ]:
data_impute_oneplant = data[data.plantID == 38]

In [ ]:
pyplot.figure(figsize=(16,12))
for i, var in enumerate(['U','V','Pout']):
    pyplot.subplot(3,1,i+1)
    pyplot.plot(data_impute_oneplant['ts'], data_impute_oneplant[var], lw=3)
    pyplot.title(var, y=0.25, loc='right', fontsize=20)

## 5. Format Data
Data are formated for REISE.

In [ ]:
data_reise = helpers.to_reise(data)

In [ ]:
data_reise.head()